!pip install nest_asyncio

!pip install fastapi uvicorn pyngrok

!pip install kss

!pip install --upgrade kss

%pip install fastapi uvicorn uvicorn[standard]

%pip install fastapi uvicorn uvicorn[standard]

!pip install webdriver-manager

In [1]:
# 2. 주피터 노트북에서 비동기 서버를 실행하기 위한 설정
import nest_asyncio
nest_asyncio.apply()

print("=" * 50)
print("FASTAPI 환경 설치 완료 및 nest_asyncio 적용 완료.")
print("=" * 50)


## 2. 일기 분석 FastAPI 서버 (KSS 완전 제거 버전)

import uvicorn
from fastapi import FastAPI, Query, HTTPException # HTTPException 추가
from typing import List, Tuple
import re
import threading
import asyncio
import time
from uvicorn.config import Config
from uvicorn.server import Server
import urllib.parse
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from datetime import datetime
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import os # Selenium 실행 중 환경 변수 설정 등 필요할 수 있으므로 추가

# Selenium 관련 경고 메시지 제거 (선택 사항)
os.environ['WDM_LOG_LEVEL'] = '0'


class PCSearchRequest(BaseModel):
    keyword: str
class VIDEOItem(BaseModel):
    name:str
    thumnail:str
    link:str
    viewCount:str
    videoDate:str
    videoAuthor:str
    videoAuthorImg:str
    videoDescript:str
class PCResponse(BaseModel):
    keyword:str
    results:List[VIDEOItem]
    time:str


# ==============================================================================
# 1. 유틸리티 Enum 및 KSS 대체 문장 분리 함수 (Regex 기반) 정의
# ==============================================================================

class Emotion:
    # 8가지 감정 이름
    EMOTION_NAMES = [
        "joy", "hope", "neutrality", "anger", 
        "sadness", "anxiety", "tiredness", "regret"
    ]
    JOY = 0; HOPE = 1; NEUTRALITY = 2; ANGER = 3
    SADNESS = 4; ANXIETY = 5; TIREDNESS = 6; REGRET = 7
    
class Depression:
    NOT_DEPRESS = 0; DEPRESS = 1
    
# KSS를 대체하는 안전한 정규 표현식 기반 문장 분리기
def split_sentences_safe(text: str) -> List[str]:
    """
    마침표, 물음표, 느낌표 뒤에 공백이 오거나 줄바꿈이 있을 때 분리합니다.
    """
    if not text: return []
    
    # 문장 끝 기호(.?!)를 기준으로 분리
    sentences = re.split(r'(?<=[.?!])\s*|\n', text)
    
    # 빈 문자열 제거 및 공백 제거
    result = [s.strip() for s in sentences if s.strip()]
    
    # 만약 텍스트가 있었으나 분리되지 않은 경우 (ex. '후회만 남네') 전체를 하나의 문장으로 반환
    if not result and text.strip():
        return [text.strip()]
            
    return result

print("[*] KSS 라이브러리 사용을 건너뛰고, Regex 기반 문장 분리기를 사용합니다.")

# ==============================================================================
# 2. Mock 서비스 정의 (일기 분석 및 챗봇)
# ==============================================================================

class MockEmotionService:
    """외부 의존성 없이 키워드 기반으로 감정 및 우울증 위험도를 모의 예측하는 서비스"""
    
    # 키워드 정의 (이전 코드와 동일)
    POSITIVE_KEYWORDS = ["좋", "행복", "기쁘", "감사", "신나", "예쁘", "사랑", "성공"]
    NEGATIVE_KEYWORDS = ["화", "슬프", "힘들", "불안", "짜증", "피곤", "후회", "억울", "죽", "절망"]
    N_ANGER_KW = ["화", "짜증", "분노", "열받"]
    N_SADNESS_KW = ["슬프", "눈물", "우울", "외롭"]
    N_ANXIETY_KW = ["불안", "걱정", "두려", "초조", "답답"]
    N_TIREDNESS_KW = ["피곤", "지치", "녹초"]
    N_REGRET_KW = ["후회", "아쉽", "밀려서"]
    P_HOPE_KW = ["할 수 있", "희망", "기대", "미래"]
    DEPRESSION_KW = ["죽", "의미 없", "사라지고 싶", "절망", "포기하"]

    def _predict_pnn(self, sent: str) -> str:
        sent_lower = sent.lower()
        if any(kw in sent_lower for kw in self.POSITIVE_KEYWORDS) and not any(kw in sent_lower for kw in self.NEGATIVE_KEYWORDS):
            return "POSITIVE"
        if any(kw in sent_lower for kw in self.NEGATIVE_KEYWORDS):
            return "NEGATIVE"
        return "NEUTRAL"

    def predict(self, sent: str) -> int:
        """문장의 8가지 감정 인덱스 (0~7)를 예측"""
        pnn_result = self._predict_pnn(sent)
        sent_lower = sent.lower()

        if pnn_result == "POSITIVE":
            if any(kw in sent_lower for kw in self.P_HOPE_KW):
                return Emotion.HOPE
            return Emotion.JOY # 기본 긍정

        elif pnn_result == "NEGATIVE":
            if any(kw in sent_lower for kw in self.N_ANGER_KW): return Emotion.ANGER
            if any(kw in sent_lower for kw in self.N_ANXIETY_KW): return Emotion.ANXIETY
            if any(kw in sent_lower for kw in self.N_TIREDNESS_KW): return Emotion.TIREDNESS
            if any(kw in sent_lower for kw in self.N_REGRET_KW): return Emotion.REGRET
            return Emotion.SADNESS

        return Emotion.NEUTRALITY

    def predict_depression(self, sent: str) -> int:
        """문장의 우울증 위험도 (0:정상, 1:우울)를 예측"""
        if any(kw in sent.lower() for kw in self.DEPRESSION_KW):
            return Depression.DEPRESS
        return Depression.NOT_DEPRESS

class MockChatbotService:
    """
    ch_kogpt2.predict() 및 ch_kobert.chat()의 동작을 모방하는 Mock 클래스입니다.
    """
    
    # /chatbot/g (KoGPT2 기반 Simple Chatbot) 응답 Mock
    def chat_g(self, sent: str) -> str:
        """/chatbot/g 엔드포인트의 응답을 모방 (단순 답변)"""
        if "안녕" in sent:
            return "안녕하세요. 저는 당신의 이야기를 들어줄 수 있는 챗봇입니다."
        if "힘들" in sent or "지치" in sent:
            return "많이 힘드시군요. 괜찮아요, 여기에 모든 것을 털어놓아도 좋습니다."
        return f"GPT2 스타일의 응답: {sent}에 대해 더 자세히 이야기해 주시겠어요?"

    # /chatbot/b (KoBERT 기반 Complex Chatbot) 응답 Mock
    def chat_b(self, sent: str) -> Tuple[str, str, str, List[float]]:
        """
        ch_kobert.chat(sentence)의 결과: answer, category, desc, softmax를 모방합니다.
        (answer만 사용됨)
        """
        category = "감정/일반"
        desc = "일반적인 대화 흐름입니다."
        softmax = [0.1] * 8 
        
        if "화가" in sent or "분노" in sent:
            answer = "화가 나시는군요. 어떤 일 때문에 그렇게 화가 나셨는지 이야기해 주실 수 있나요?"
        elif "우울" in sent or "죽고" in sent:
            answer = "현재 많이 힘들어 보이세요. 전문 상담사와 연결해 드릴까요?"
        else:
            answer = f"KoBERT 스타일의 응답: 당신의 {sent}에 대해 제가 드릴 수 있는 구체적인 조언이 있을까요?"
            
        return answer, category, desc, softmax

# ==============================================================================
# 3. 서비스 인스턴스 및 FastAPI 앱 초기화
# ==============================================================================
emotion_service = MockEmotionService()
chatbot_service = MockChatbotService() # 챗봇 서비스 인스턴스 추가
app = FastAPI(title="FastAPI 통합 서버 (일기 분석, 챗봇, 크롤링)", version="1.0.2")

# --------------------------------------------------------
# 1. CORS 미들웨어 설정
# --------------------------------------------------------
origins = [
    "http://192.168.0.9:8083",
    "http://192.168.0.5:8083",
    "http://192.168.0.1:8083",
    "http://192.168.0.21:8083",
    "http://localhost:8083",
    "http://127.0.0.1:8083",
    # 필요하다면 모든 출처를 허용하려면 "*"을 사용하지만, 보안상 권장되지는 않습니다.
]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,       # 허용할 출처 목록
    allow_credentials=True,      # 쿠키와 같은 자격 증명 허용
    allow_methods=["*"],         # 모든 HTTP 메서드 (GET, POST, OPTIONS 등) 허용
    allow_headers=["*"],         # 모든 HTTP 헤더 허용
)

# ==============================================================================
# 4. API 엔드포인트 정의
# ==============================================================================

@app.get("/")
async def hello():
    """서버 상태 확인 엔드포인트"""
    return {"message": "Integrated Server (Diary, Chatbot, Crawler) is running 🚀 - FastAPI"}


@app.get("/diary")
async def classify_emotion_diary(
    s: str = Query(..., description="일기 전체 텍스트 블록 (예: '오늘도 너무 피곤해서 후회만 남네. 이대 합격해서 기뻤던 희망이 사라지고 싶어.')")
):
    """일기 전체에 대한 감정 분포 (8가지) 및 우울증 위험도 분석"""
    diary_text = s.strip()    
    # 디코딩 실행 (FastAPI는 기본적으로 쿼리 매개변수를 자동 디코딩하지만, 명시적 디코딩 로직 유지)
    decoded_string = urllib.parse.unquote(diary_text)
    print(f"인코딩된 문자열: {diary_text}")
    print(f"디코딩된 문자열: {decoded_string}")
    
    predict_counts = [0] * 8 
    total_cnt = 0.0
    dep_cnt = 0
    
    # 1. 문장 분리 (안전한 Regex 기반 함수 사용)
    sentences = split_sentences_safe(decoded_string)
    
    # 2. 문장별 분석 및 카운트
    for sent in sentences:
        
        try:
            emotion_result = emotion_service.predict(sent)
            
            if 0 <= emotion_result < 8:
                predict_counts[emotion_result] += 1
                total_cnt += 1
            else:
                predict_counts[Emotion.NEUTRALITY] += 1
                total_cnt += 1
                
            if emotion_service.predict_depression(sent) == Depression.DEPRESS:
                dep_cnt += 1
                
        except Exception as e:
            # 예측 중 오류가 발생하면 해당 문장 건너뛰기
            print(f"[*] CRITICAL ERROR processing sentence (Skipping): {sent}. Error: {e}")
            continue

    # 3. 결과 계산 (퍼센트 및 우울증 위험도)
    results = {}
    dep_predict = 0.0
    
    if total_cnt > 0:
        for i, name in enumerate(Emotion.EMOTION_NAMES):
            percentage = round((predict_counts[i] / total_cnt) * 100, 2)
            results[name] = percentage
        
        dep_predict = round((dep_cnt / total_cnt) * 100, 2)
    else:
        # 입력이 없거나 문장이 분리되지 않았을 경우
        results = {name: 100.0 if name == "neutrality" else 0.0 for name in Emotion.EMOTION_NAMES}

    # 4. 최종 결과 반환
    return results | {"depression": dep_predict}


# ==============================================================================
# Chatbot 엔드포인트 추가 (요청하신 부분)
# ==============================================================================

# /chatbot/g?s=입력 (Flask의 reactChatbotV1 대응)
@app.get("/chatbot/g")
async def reactChatbotV1(
    s: str = Query(None, alias="s", description="챗봇에게 전달할 사용자 입력 문장")
):
    """KoGPT2 기반 Simple Chatbot Mock (Flask의 reactChatbotV1 대응)"""
    sentence = s.strip() if s else ""
    
    if not sentence:
        return {
            "answer": "듣고 있어요. 더 말씀해주세요~ (끄덕끄덕)"
        }

    # ch_kogpt2.predict(sentence) 모방
    answer = chatbot_service.chat_g(sentence)
    
    return {
        "answer": answer
    }

# /chatbot/b?s=입력 (Flask의 reactChatbotV2 대응)
@app.get("/chatbot/b")
async def reactChatbotV2(
    s: str = Query(None, alias="s", description="챗봇에게 전달할 사용자 입력 문장")
):
    """KoBERT 기반 Complex Chatbot Mock (Flask의 reactChatbotV2 대응)"""
    sentence = s.strip() if s else ""

    if not sentence:
        # 입력이 비었을 때의 기본 응답
        return {
            "answer": "듣고 있어요. 더 말씀해주세요~ (끄덕끄덕)"
        }

    # ch_kobert.chat(sentence) 모방 (answer, category, desc, softmax 반환)
    answer, category, desc, softmax = chatbot_service.chat_b(sentence)
    
    # 원본 Flask 코드와 동일하게 'answer'만 반환합니다.
    return {
        "answer": answer
    }


# ==============================================================================
# /youtube/search 엔드포인트 (기존 코드 유지, Selenium 포함)
# ==============================================================================
@app.post("/youtube/search",response_model=PCResponse)
def youtube_search_crawler(req: PCSearchRequest, max_videos=10):
    """
    유튜브 검색 결과에서 영상 정보를 크롤링합니다.
    (주의: 주피터 환경이나 headless 환경에서 Selenium/WebDriverManager는 실패할 가능성이 높습니다.)
    """
    search_query = ''
    
    if req.keyword == '분노':
        search_query = "분노 가라앉히는 음악"
    elif req.keyword == '불안':
        search_query = "불안할때 듣는 노래"
    elif req.keyword == '슬픔':
        search_query = "슬플때 듣는 노래"
    elif req.keyword == '기쁨':
        search_query = "기쁠때 듣는 노래"
    elif req.keyword == '후회':
        search_query = "후회할때 듣는노래"
    elif req.keyword == '희망':
        search_query = "희망찬 브금"
    elif req.keyword == '중립':
        search_query = "머릿속이 복잡할때 듣는 노래"
    elif req.keyword == '피로':
        search_query = "지치고 힘들때 듣는 노래"
    elif req.keyword == '우울':
        search_query = "우울함을 달래줄 노래"
    else :
        search_query = "후회할때 듣는노래"

    # 1. WebDriver 설정 및 브라우저 열기
    print("🚀 WebDriver를 설정하고 브라우저를 엽니다...")
    options = webdriver.ChromeOptions()
    # options.add_argument('headless') # 백그라운드에서 실행하려면 주석 해제
    
    try:
        # 크롬 드라이버 자동 관리
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=options)
    except Exception as e:
        # WebDriver 설정 실패 시, 오류 반환
        raise HTTPException(status_code=500, detail=f"WebDriver 설정 오류: {e}")
    
    # 유튜브 검색 URL 생성
    base_url = "https://www.youtube.com/results?search_query="
    search_url = base_url + search_query.replace(' ', '+')
    
    driver.get(search_url)
    time.sleep(3) # 페이지 로딩 대기

    # 2. 크롤링을 위한 HTML 소스 가져오기
    print("🔍 HTML 소스를 가져옵니다...")
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # 3. 영상 정보 추출
    print("✂️ 영상 정보를 추출합니다...")
    items = []
    
    video_elements = soup.select('ytd-video-renderer') 
    
    for i, video in enumerate(video_elements):
        if len(items) >= max_videos:
            break
            
        # 제목 및 링크 추출
        title_tag = video.select_one('#video-title')
        
        if title_tag:
            title = title_tag.text.strip()
            # /watch?v=... 형태의 링크를 전체 URL로 변환
            link = "https://www.youtube.com" + title_tag.get('href', '')
            
        else:
            continue
        
        # 조회수 및 날짜 추출
        view_info = video.select('#metadata-line > span.inline-metadata-item')
        view_count_element = view_info[0] if len(view_info)>=1 else None
        viewCount = view_count_element.text.strip() if view_count_element else 'N/A'

        date_element = view_info[1] if len(view_info)>=2 else None
        videoDate = date_element.text.strip() if date_element else 'N/A'

        # 채널 이미지 추출
        author_img = video.select_one('#channel-thumbnail > yt-img-shadow > img')
        author_img_url = author_img.get('src', author_img.get('data-src', '')) if author_img else 'N/A'

        # 채널 이름 추출
        author_element = video.select_one('#channel-name a')
        videoAuthor = author_element.text.strip() if author_element else 'N/A'

        # 영상 설명 추출
        descript_element = video.select_one('.metadata-snippet-container')
        videoDescript = descript_element.text.strip() if descript_element else 'N/A'
        
        # 썸네일 이미지 URL 추출
        thumbnail_img = video.select_one('#thumbnail > yt-image > img')
        thumbnail_url = thumbnail_img.get('src', thumbnail_img.get('data-src', '')) if thumbnail_img else 'N/A'

        # 정보 저장
        items.append(
            VIDEOItem(
                name=title,
                thumnail=thumbnail_url,
                link=link,
                viewCount=viewCount,
                videoDate=videoDate,
                videoAuthor=videoAuthor,
                videoAuthorImg=author_img_url,
                videoDescript=videoDescript
            )
        )

    # 4. 브라우저 종료
    print("🛑 브라우저를 종료합니다.")
    driver.quit()

    if not items:
        raise HTTPException(status_code = 404, detail="검색 결과 없음")

    return PCResponse(
        keyword=req.keyword,
        results=items,
        time=datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    )

# ==============================================================================
# 5. 서버 실행 (주피터 최적화)
# ==============================================================================

async def serve_fastapi():
    # host="0.0.0.0" 설정을 통해 192.168.0.5의 모든 IP(내부/외부) 접근을 허용합니다.
    config = Config(app, host="0.0.0.0", port=8000, log_level="warning") 
    server = Server(config=config)
    await server.serve()

def start_server_thread():
    print("=" * 50)
    print("[*] FastAPI 통합 서버를 백그라운드에서 시작합니다.")
    print("[*] API 문서: http://192.168.0.20:8000/docs (혹은 127.0.0.1:8000)")
    print("=" * 50)
    
    # 서버 실행
    try:
        asyncio.run(serve_fastapi()) 
    except RuntimeError:
        print("Server loop already running. Using existing loop.")

if 'server_thread' not in locals() or not locals()['server_thread'].is_alive():
    server_thread = threading.Thread(target=start_server_thread, daemon=True)
    server_thread.start()
    time.sleep(1) 
    print("[*] 서버 스레드가 시작되었습니다. 브라우저에서 /docs 페이지를 확인하세요.")
else:
    print("[*] 서버가 이미 실행 중인 것 같습니다. 다시 시작하려면 커널을 재시작해 주세요.")

FASTAPI 환경 설치 완료 및 nest_asyncio 적용 완료.
[*] KSS 라이브러리 사용을 건너뛰고, Regex 기반 문장 분리기를 사용합니다.
[*] FastAPI 통합 서버를 백그라운드에서 시작합니다.
[*] API 문서: http://192.168.0.20:8000/docs (혹은 127.0.0.1:8000)
[*] 서버 스레드가 시작되었습니다. 브라우저에서 /docs 페이지를 확인하세요.
